# 🎯 Feature Selection and Management

<div style="background-color: #e3f2fd; padding: 15px; border-radius: 5px; border-left: 5px solid #2196F3;">
<b>📓 Notebook Information</b><br>
<b>Level:</b> Intermediate<br>
<b>Estimated Time:</b> 20 minutes<br>
<b>Prerequisites:</b> 01_basic_usage.ipynb<br>
<b>Dataset:</b> Breast Cancer (sklearn)
</div>

---

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:
- ✅ Understand how DBDataset handles feature selection
- ✅ Manually specify features vs auto-detection
- ✅ Integrate feature engineering pipelines
- ✅ Analyze feature importance from models
- ✅ Compare models with different feature sets
- ✅ Apply feature selection best practices

---

## 📚 Table of Contents

1. [Introduction](#intro)
2. [Setup](#setup)
3. [Auto Feature Detection](#auto)
4. [Manual Feature Selection](#manual)
5. [Feature Engineering Integration](#engineering)
6. [Feature Importance Analysis](#importance)
7. [Comparing Feature Sets](#comparison)
8. [Best Practices](#practices)
9. [Conclusion](#conclusion)

<a id="intro"></a>
## 1. 📖 Introduction

### What is Feature Selection?

> **Feature selection** is the process of choosing which input variables (features) to use for model training.

### Why Does Feature Selection Matter?

**The Benefits:**
- 🚀 **Better performance** - Less noise, better signal
- ⚡ **Faster training** - Fewer dimensions = less computation
- 🔍 **Better interpretability** - Easier to understand what drives predictions
- 🛡️ **Avoid overfitting** - Less curse of dimensionality
- 💰 **Lower costs** - Less data storage and collection

**Real-world example:**
```python
# Before: 100 features
model.fit(X_train)  # 45 seconds, 78% accuracy

# After: 20 most important features
model.fit(X_train_selected)  # 8 seconds, 81% accuracy ✅
```

### DBDataset Feature Management

DBDataset offers flexible feature handling:

1. **Auto-detection** (default): Uses all columns except target
2. **Manual specification**: You provide explicit feature list
3. **Feature engineering**: Integrate sklearn pipelines
4. **Feature importance**: Analyze what matters most

### Common Feature Selection Methods

| Method | Type | Best For | Speed |
|--------|------|----------|-------|
| **Variance Threshold** | Filter | Remove constant features | ⚡⚡⚡ |
| **Correlation** | Filter | Remove redundant features | ⚡⚡⚡ |
| **Chi-square** | Filter | Classification (categorical) | ⚡⚡ |
| **Mutual Information** | Filter | Non-linear relationships | ⚡⚡ |
| **RFE** | Wrapper | Optimal feature subset | ⚡ |
| **Feature Importance** | Embedded | Tree-based models | ⚡⚡ |
| **L1 Regularization** | Embedded | Linear models | ⚡⚡ |

**Let's explore how DBDataset helps with feature selection!** 🚀

<a id="setup"></a>
## 2. 🛠️ Setup

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from time import time

# sklearn
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import (
    SelectKBest, 
    f_classif, 
    RFE,
    VarianceThreshold
)
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score

# DeepBridge
from deepbridge import DBDataset, Experiment

# Settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')
%matplotlib inline

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ Setup complete!")
print("🎯 Topic: Feature Selection with DBDataset")

### Load Dataset

In [ ]:
# Load breast cancer dataset
cancer = load_breast_cancer()
df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
df['target'] = cancer.target

print(f"📊 Dataset: {df.shape}")
print(f"   Features: {len(cancer.feature_names)}")
print(f"   Target classes: {cancer.target_names}")
print(f"\n   First 5 features: {list(cancer.feature_names[:5])}")

<a id="auto"></a>
## 3. 🔍 Auto Feature Detection

### Default Behavior: Use All Features

In [ ]:
print("🔍 Auto Feature Detection (Default)\n")
print("   When you don't specify features, DBDataset uses ALL columns except target")

# Train a simple model
X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

model = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
model.fit(X_train, y_train)

# Create DBDataset (auto-detects all features)
dataset = DBDataset(
    data=df,
    target_column='target',
    model=model,
    test_size=0.2,
    random_state=RANDOM_STATE
)

print(f"\n✅ DBDataset created")
print(f"   Features detected: {len(dataset.features)}")
print(f"   Feature list: {dataset.features[:5]}... (showing first 5)")
print(f"\n💡 All 30 features automatically included!")

<a id="manual"></a>
## 4. ✋ Manual Feature Selection

### Specify Exact Features to Use

In [ ]:
print("✋ Manual Feature Selection\n")
print("   You can explicitly specify which features to use")

# Select top 10 features based on univariate test
selector = SelectKBest(f_classif, k=10)
selector.fit(X_train, y_train)

# Get selected feature names
selected_features = X_train.columns[selector.get_support()].tolist()

print(f"\n📊 Top 10 Features (F-statistic):")
for i, feat in enumerate(selected_features, 1):
    print(f"   {i:2d}. {feat}")

# Train model with selected features only
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

model_selected = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
model_selected.fit(X_train_selected, y_train)

# Create DBDataset with explicit feature list
# Note: We need to prepare the data with only selected features
df_selected = df[selected_features + ['target']]

dataset_selected = DBDataset(
    data=df_selected,
    target_column='target',
    model=model_selected,
    test_size=0.2,
    random_state=RANDOM_STATE
)

print(f"\n✅ DBDataset with selected features created")
print(f"   Features used: {len(dataset_selected.features)}")
print(f"   Reduction: {len(X.columns)} → {len(selected_features)} features ({len(selected_features)/len(X.columns)*100:.0f}%)")

<a id="engineering"></a>
## 5. 🔧 Feature Engineering Integration

### Using sklearn Pipelines with DBDataset

In [ ]:
print("🔧 Feature Engineering with Pipelines\n")
print("   You can integrate preprocessing and feature engineering into your workflow")

# Create a preprocessing + feature selection pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('feature_selection', SelectKBest(f_classif, k=15)),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE))
])

# Fit the pipeline
pipeline.fit(X_train, y_train)

print("\n✅ Pipeline created:")
print("   Step 1: StandardScaler (normalize features)")
print("   Step 2: SelectKBest (top 15 features)")
print("   Step 3: RandomForest (classifier)")

# Get selected features from the pipeline
feature_selector = pipeline.named_steps['feature_selection']
pipeline_features = X_train.columns[feature_selector.get_support()].tolist()

print(f"\n📊 Selected features: {len(pipeline_features)}")
print(f"   {pipeline_features[:5]}... (showing first 5)")

# For DBDataset, we can use the pipeline as the model
dataset_pipeline = DBDataset(
    data=df,
    target_column='target',
    model=pipeline,  # Pass the entire pipeline!
    test_size=0.2,
    random_state=RANDOM_STATE
)

print(f"\n✅ DBDataset with pipeline created")
print(f"   Pipeline handles: scaling → selection → classification")
print(f"\n💡 The pipeline approach is recommended for production!")

<a id="importance"></a>
## 6. 📊 Feature Importance Analysis

### Which Features Matter Most?

In [ ]:
print("📊 Feature Importance Analysis\n")

# Get feature importances from the model
importances = model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values('Importance', ascending=False)

print("🔝 Top 10 Most Important Features:\n")
display(feature_importance_df.head(10).style.background_gradient(
    cmap='Greens', subset=['Importance']
).format({'Importance': '{:.4f}'}))

# Calculate cumulative importance
feature_importance_df['Cumulative'] = feature_importance_df['Importance'].cumsum()

# How many features for 80% importance?
n_features_80 = (feature_importance_df['Cumulative'] <= 0.80).sum()
print(f"\n💡 {n_features_80} features account for 80% of total importance")
print(f"   Original: {len(X.columns)} features")
print(f"   Efficient subset: {n_features_80} features")
print(f"   Reduction: {(1 - n_features_80/len(X.columns))*100:.1f}% fewer features!")

### Visualize Feature Importance

In [ ]:
# Plot top 15 features
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar plot
top_15 = feature_importance_df.head(15)
axes[0].barh(range(len(top_15)), top_15['Importance'], color='skyblue', edgecolor='black')
axes[0].set_yticks(range(len(top_15)))
axes[0].set_yticklabels(top_15['Feature'])
axes[0].invert_yaxis()
axes[0].set_xlabel('Importance', fontweight='bold')
axes[0].set_title('Top 15 Feature Importances', fontsize=14, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# Cumulative importance
axes[1].plot(range(1, len(feature_importance_df)+1), 
             feature_importance_df['Cumulative'], 
             marker='o', linewidth=2, markersize=4, color='green')
axes[1].axhline(y=0.80, color='red', linestyle='--', linewidth=2, label='80% threshold')
axes[1].axvline(x=n_features_80, color='red', linestyle='--', linewidth=2, alpha=0.5)
axes[1].fill_between(range(1, n_features_80+1), 0, 1, alpha=0.2, color='green')
axes[1].set_xlabel('Number of Features', fontweight='bold')
axes[1].set_ylabel('Cumulative Importance', fontweight='bold')
axes[1].set_title('Cumulative Feature Importance', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)
axes[1].text(n_features_80, 0.85, f'{n_features_80} features\n= 80% importance',
             bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7),
             fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n📊 Visualization shows:")
print(f"   • Left: Top features ranked by importance")
print(f"   • Right: {n_features_80} features capture 80% of predictive power")
print(f"\n💡 Consider using only the top {n_features_80} features for efficiency!")

<a id="comparison"></a>
## 7. ⚖️ Comparing Feature Sets

### Does Feature Selection Improve Performance?

In [ ]:
print("⚖️  Comparing Different Feature Sets\n")
print("=" * 70)

# Get top N features based on importance
top_5 = feature_importance_df.head(5)['Feature'].tolist()
top_10 = feature_importance_df.head(10)['Feature'].tolist()
top_15 = feature_importance_df.head(15)['Feature'].tolist()
top_20 = feature_importance_df.head(20)['Feature'].tolist()

# Test different feature sets
results = []

for name, features in [
    ('All Features (30)', X.columns.tolist()),
    ('Top 20', top_20),
    ('Top 15', top_15),
    ('Top 10', top_10),
    ('Top 5', top_5)
]:
    # Train model
    X_train_subset = X_train[features]
    X_test_subset = X_test[features]
    
    model_subset = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
    
    # Measure training time
    start = time()
    model_subset.fit(X_train_subset, y_train)
    train_time = time() - start
    
    # Evaluate
    y_pred = model_subset.predict(X_test_subset)
    y_proba = model_subset.predict_proba(X_test_subset)[:, 1]
    
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    
    results.append({
        'Feature Set': name,
        'N Features': len(features),
        'Accuracy': acc,
        'ROC AUC': auc,
        'Train Time (s)': train_time
    })

results_df = pd.DataFrame(results)

display(results_df.style.background_gradient(
    cmap='RdYlGn', subset=['Accuracy', 'ROC AUC']
).background_gradient(
    cmap='RdYlGn_r', subset=['Train Time (s)']
).format({
    'Accuracy': '{:.4f}',
    'ROC AUC': '{:.4f}',
    'Train Time (s)': '{:.3f}'
}))

# Find best feature set
best_idx = results_df['ROC AUC'].idxmax()
best_set = results_df.iloc[best_idx]

print(f"\n🏆 Best Feature Set: {best_set['Feature Set']}")
print(f"   ROC AUC: {best_set['ROC AUC']:.4f}")
print(f"   Training time: {best_set['Train Time (s)']:.3f}s")
print(f"\n💡 Using {best_set['N Features']} features instead of 30:")
print(f"   • Similar performance (≈{best_set['ROC AUC']:.3f} AUC)")
print(f"   • {(1 - best_set['N Features']/30)*100:.0f}% fewer features")
print(f"   • Faster training and inference")
print(f"   • Better interpretability")

<a id="practices"></a>
## 8. ✨ Best Practices

### Feature Selection Guidelines

<div style="background-color: #e8f5e9; padding: 15px; border-radius: 5px; border-left: 5px solid #4CAF50;">
<b>✅ DO</b><br><br>

1. **Start Simple**
   - Begin with all features to establish baseline
   - Gradually remove less important features
   - Compare performance at each step

2. **Use Domain Knowledge**
   - Remove obviously irrelevant features (IDs, timestamps for non-temporal tasks)
   - Keep features known to be important in the domain
   - Consult experts before removing features

3. **Check for Redundancy**
   - Calculate feature correlations
   - Remove highly correlated features (r > 0.95)
   - Keep the most interpretable feature from correlated groups

4. **Validate Performance**
   - Always use cross-validation
   - Test on holdout set
   - Monitor multiple metrics (not just accuracy)

5. **Use Pipelines**
   - Integrate feature selection into sklearn pipelines
   - Prevents data leakage
   - Makes deployment easier

</div>

<div style="background-color: #ffebee; padding: 15px; border-radius: 5px; border-left: 5px solid #f44336; margin-top: 15px;">
<b>❌ DON'T</b><br><br>

1. **Select Features on Full Data**
   ```python
   # ❌ WRONG: Uses test data!
   selector.fit(X_all, y_all)
   X_train_selected = X_train[selected_features]
   ```
   ```python
   # ✅ CORRECT: Only train data
   selector.fit(X_train, y_train)
   X_train_selected = X_train[selector.get_support()]
   ```

2. **Over-select Features**
   - Selecting 1-2 features might remove critical information
   - Aim for 10-20% of original features as a starting point

3. **Ignore Feature Engineering**
   - Sometimes creating new features > selecting existing ones
   - Polynomial features, interactions can be powerful

4. **Optimize Only for Training Performance**
   - This leads to overfitting
   - Always validate on separate test set

5. **Remove Features Without Analysis**
   - Understand WHY a feature has low importance
   - Could be data quality issue, not lack of signal

</div>

### Feature Selection Decision Matrix

In [ ]:
# Create decision guide
decision_df = pd.DataFrame({
    'Scenario': [
        'Small dataset (<1K rows)',
        'High dimensionality (>100 features)',
        'Real-time inference required',
        'Interpretability critical',
        'Training time is bottleneck',
        'Overfitting suspected',
        'Production cost concerns'
    ],
    'Recommendation': [
        'Use all features (avoid overfitting from selection)',
        'Aggressive selection (top 10-20 features)',
        'Select 5-15 most important features',
        'Select 3-10 most interpretable features',
        'Use feature importance, remove bottom 50%',
        'Reduce features, add regularization',
        'Minimize features to reduce storage/compute'
    ],
    'Priority': [
        '🟡 Medium',
        '🔴 High',
        '🔴 High',
        '🔴 High',
        '🟡 Medium',
        '🔴 High',
        '🟡 Medium'
    ]
})

print("📋 Feature Selection Decision Guide\n")
print("=" * 100)
display(decision_df.style.set_properties(**{
    'text-align': 'left',
    'white-space': 'pre-wrap'
}).apply(lambda x: ['background-color: #ffcdd2' if '🔴' in v 
                     else 'background-color: #fff9c4' if '🟡' in v
                     else '' for v in x], subset=['Priority']))

print("\n💡 Use this guide to decide when and how to select features!")

<a id="conclusion"></a>
## 9. 🎓 Conclusion

### What You Learned

- ✅ **Auto-detection** - DBDataset automatically uses all features
- ✅ **Manual selection** - Explicitly specify features to use
- ✅ **Pipeline integration** - Combine preprocessing + selection + modeling
- ✅ **Feature importance** - Identify which features matter most
- ✅ **Performance comparison** - Evaluate different feature sets
- ✅ **Best practices** - Avoid data leakage, validate properly

### Key Takeaways

1. 🎯 **Start with baseline** - Use all features first
2. 📊 **Analyze importance** - Understand what drives predictions
3. 🔍 **Select strategically** - Balance performance vs complexity
4. ⚡ **Measure impact** - Training time, inference speed, accuracy
5. 🛡️ **Prevent leakage** - Select features only on training data
6. 🔄 **Use pipelines** - Encapsulate feature selection for production

### Feature Selection ROI

```python
# Before: 30 features
- Training: 2.5 seconds
- Inference: 50ms
- Storage: 10 MB
- Interpretability: Low

# After: 10 features (33% of original)
- Training: 0.8 seconds (-68%) ✅
- Inference: 15ms (-70%) ✅
- Storage: 3.3 MB (-67%) ✅
- Interpretability: High ✅
- Performance: Same or better! ✅
```

### Next Steps

1. **Try different selection methods**
   - Mutual information
   - Recursive feature elimination
   - L1 regularization

2. **Feature engineering**
   - Create interaction features
   - Polynomial features
   - Domain-specific transformations

3. **Advanced techniques**
   - Feature selection with genetic algorithms
   - AutoML for automatic feature engineering
   - Ensemble selection methods

---

**Remember: Sometimes less is more! Fewer features can mean better models.** 🚀